In [1]:
from langchain_community.tools import ArxivQueryRun , WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper , ArxivAPIWrapper

/Users/akashjain/Desktop/LangGraph/venvLanggraph/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# CUSTOM TOOL ?

In [7]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results = 1 , doc_content_chars_max = 250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [8]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results = 1 , doc_content_chars_max = 250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

# RAG

In [12]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [25]:
import os

from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [26]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
embeddings


GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x1205cea90>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [27]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")

docs = loader.load()

documents = RecursiveCharacterTextSplitter( chunk_size = 1000 , chunk_overlap = 200 ).split_documents(docs)

vectordb = FAISS.from_documents(documents , embeddings)

retriever = vectordb.as_retriever()


In [31]:
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12381c880>, search_kwargs={})

In [28]:
content = retriever.invoke("hi how are you")
content[0].page_content

'Get started with LangSmith | 🦜️🛠️ LangSmith'

# converting this retriever into a tool

In [29]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever , "langsmith-searc" , "Hey search any info regarding langsmith")

In [30]:
retriever_tool

Tool(name='langsmith-searc', description='Hey search any info regarding langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x107bfa550>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12381c880>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x107bfa9d0>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12381c880>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separat

In [32]:
tools = [arxiv , wiki , retriever_tool]
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/akashjain/Desktop/LangGraph/venvLanggraph/lib/python3.9/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-searc', description='Hey search any info regarding langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x107bfa550>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at

In [33]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-pro" , temperature = 0 , max_tokens=1000)
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-pro', google_api_key=SecretStr('**********'), temperature=0.0, max_output_tokens=1000, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x1240c7ee0>, default_metadata=())

In [42]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [60]:
#there is hub in langchain which has predefined templated
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")

print(type(prompt) , prompt.messages)

<class 'langchain_core.prompts.chat.ChatPromptTemplate'> [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')]


In [57]:
from langchain_core.messages import HumanMessage , AIMessage , ToolMessage

chain = prompt | llm | parser
response = chain.invoke( 
    {
        "input" : "Tell me about langsmith",
         "agent_scratchpad": [
             AIMessage(content="I will tell you things related to langchain'")
         ]
    }
)
print(response)

# In LangChain Agents, agent_scratchpad is where the agent "writes down" its thoughts, actions, and observations — just like how a human might jot notes while solving a problem.
# LangChain’s agent framework uses agent_scratchpad to store the running reasoning history.

LangSmith is a platform designed to help developers build, debug, test, evaluate, and monitor LLM applications.  It's not directly *part* of LangChain, but it's a very useful tool for anyone working with LangChain, or other LLM frameworks.  Think of it as a powerful toolkit for enhancing your LLM development workflow.

Here's a breakdown of what LangSmith offers:

* **Tracing and Debugging:**  LangSmith captures the full execution flow of your LLM application, including prompts, responses, chain executions, and tool calls. This detailed trace allows you to pinpoint issues, understand the flow of data, and identify areas for improvement.  It's invaluable for debugging complex chains and understanding why your application is behaving a certain way.

* **Experiment Tracking:**  LangSmith helps you manage and compare different versions of your prompts, chains, and models.  You can track metrics like cost, latency, and performance to optimize your application for efficiency and effectivenes

# AGENTS

At the end we have to convert these tools into agent

In [62]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm , tools , prompt)

agent

# any llm can be used
# what is runnable binding

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [63]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent , tools  = tools , verbose = True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [64]:
agent_executor.invoke(
    {
        "input" : "Tell me about langsmith"
    }
)



> Entering new AgentExecutor chain...

Invoking: `langsmith-searc` with `{'query': 'langsmith'}`


Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback on your data.Prompt Engineerin

{'input': 'Tell me about langsmith',
 'output': 'LangSmith is a platform designed for building, monitoring, and evaluating production-grade LLM applications. It offers features for observability, evaluation, prompt engineering, and deployment.  It helps developers understand LLM application behavior, iterate on prompts, and improve performance. It integrates with LangChain and LangGraph, enabling tracing with a single environment variable.  It also provides tools for creating evaluations, analyzing results, and collecting human feedback.'}

In [65]:
agent_executor.invoke(
    {
        "input" : "Tell me about abstract of paper [1706,03762] titled Attetion is all you need"
    }
)



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cThe abstract of the paper "Attention is All You Need" is:  The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality and significantly more parallelizable and require less time to train. Our model achieves 28.4 BLEU on the WMT 2014

{'input': 'Tell me about abstract of paper [1706,03762] titled Attetion is all you need',
 'output': 'The abstract of the paper "Attention is All You Need" is:  The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality and significantly more parallelizable and require less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 

In [66]:
agent_executor.invoke(
    {
        "input" : "Tell me about ML and DL"
    }
)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine Learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': 'Deep Learning'}`


Page: Deep learning
Summary: Deep learning is a subset of machine learning that focuses on utilizing multilayered neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from bioloMachine learning (ML) is a field of study in artificial intelligence concerned with the development of algorithms that can learn from and make decisions on data.  Deep learning is a subfield of machine learning that uses multi-layered artificial neural networks to analyze data.  It's called "deep" because these networks have many layers,

{'input': 'Tell me about ML and DL',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development of algorithms that can learn from and make decisions on data.  Deep learning is a subfield of machine learning that uses multi-layered artificial neural networks to analyze data.  It\'s called "deep" because these networks have many layers, allowing them to learn complex patterns and representations from data.  Deep learning has been successfully applied to various tasks, including image recognition, natural language processing, and speech recognition.'}

# ReAct AGENT 

In [68]:
from langchain.agents import create_react_agent

# agent = create_react_agent(llm , tools , prompt)

# agent

In [69]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent , tools  = tools , verbose = True , handle_parsing_errors=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [70]:
agent_executor.invoke(
    {
        "input" : "Tell me about abstract of paper [1706,03762] titled Attetion is all you need"
    }
)



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cThe abstract of the paper "Attention is All You Need" is:  The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality and significantly more parallelizable and require less time to train. Our model achieves 28.4 BLEU on the WMT 2014

{'input': 'Tell me about abstract of paper [1706,03762] titled Attetion is all you need',
 'output': 'The abstract of the paper "Attention is All You Need" is:  The dominant sequence transduction models are based on complex recurrent or convolutional neural networks that include an encoder and a decoder. The best performing models also connect the encoder and decoder through an attention mechanism. We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality and significantly more parallelizable and require less time to train. Our model achieves 28.4 BLEU on the WMT 2014 English-to-German translation task, improving over the existing best results, including ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task, our model establishes a new single-model state-of-the-art BLEU score of 